In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install pandas numpy nltk Pillow requests tensorflow

In [ ]:
import pandas as pd
import numpy as np
import requests
import json
from io import BytesIO
from PIL import Image, ImageEnhance
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import math
import pickle
from collections import defaultdict
from numpy.linalg import norm

# Ensure necessary NLTK datasets are downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Initialize the ResNet model globally
resnet_model = ResNet50(weights='imagenet', include_top=False)
model = Model(inputs=resnet_model.input, outputs=resnet_model.output)

def preprocess_and_extract_features(image_url):
    """Preprocess an image from a URL and extract features using ResNet50."""
    try:
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content)).convert('RGB')
        image = image.resize((224, 224))
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(1.5)
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(1.1)
        image_array = np.array(image)
        image_array = preprocess_input(image_array)
        image_array = np.expand_dims(image_array, axis=0)
        features = model.predict(image_array)
        return (image_url, features.reshape(features.shape[0], -1).tolist())
    except Exception as e:
        print(f"Error processing image from URL {image_url}: {e}")
        return (image_url, None)

def preprocess_text(text):
    """Preprocess text data."""
    text = str(text).lower().translate(str.maketrans('', '', string.punctuation))
    tokens = nltk.word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    return ' '.join(lemmatized_tokens)

def compute_tf(text):
    """Compute term frequency for a document."""
    tf_text = defaultdict(int)
    words = text.split()
    for word in words:
        tf_text[word] += 1
    return {word: count / len(words) for word, count in tf_text.items()}

def compute_idf(documents):
    """Compute inverse document frequency across a corpus."""
    idf_dict = defaultdict(int)
    N = len(documents)
    for document in documents:
        unique_words = set(document.split())
        for word in unique_words:
            idf_dict[word] += 1
    return {word: math.log(N / (count + 1)) for word, count in idf_dict.items()}

def compute_tfidf(documents):
    """Compute TF-IDF scores for all documents in a corpus."""
    idf_dict = compute_idf(documents)
    return [{word: tf * idf_dict[word] for word, tf in compute_tf(document).items()} for document in documents]

def save_data(data, file_name):
    """Serialize data to a file."""
    with open(file_name, 'wb') as file:
        pickle.dump(data, file)

if __name__ == "__main__":
    # Example DataFrame loading and processing
    df = pd.read_csv('/content/drive/MyDrive/IR_Assignments/Assignment_2/A2_Data.csv')
    df['Image'] = df['Image'].apply(lambda x: json.loads(x.replace("'", "\""))[0] if x else None)

    # Process images and text
    image_features_with_url = [preprocess_and_extract_features(url) for url in df['Image']]


    # Save the results
    save_data(image_features_with_url, '/content/drive/MyDrive/IR_Assignments/Assignment_2/image_features.pkl')

    composite_data = []

    for index, row in df.iterrows():
        image_url = row['Image']
        review_text = row['Review Text']

        # Process image and text
        image_features = preprocess_and_extract_features(image_url)
        preprocessed_review = preprocess_text(review_text)
        tfidf_scores = compute_tfidf([preprocessed_review])[0]

        composite_data.append({
            'image_url': image_url,
            'preprocessed_review': preprocessed_review,
            'tfidf_score': tfidf_scores
        })

    save_data(composite_data, '/content/drive/MyDrive/IR_Assignments/Assignment_2/composite_data.pkl')


In [ ]:
import pickle

def load_data(file_name):
    """Load serialized data from a file."""
    with open(file_name, 'rb') as file:
        return pickle.load(file)

# Specify the paths to your files
image_features_path = '/content/drive/MyDrive/IR_Assignments/Assignment_2/image_features.pkl'
processed_reviews_path = '/content/drive/MyDrive/IR_Assignments/Assignment_2/processed_reviews.pkl'
tfidf_scores_path = '/content/drive/MyDrive/IR_Assignments/Assignment_2/tfidf_scores.pkl'


# Load the data
image_features = load_data(image_features_path)
preprocessed_reviews = load_data(processed_reviews_path)
tfidf_scores = load_data(tfidf_scores_path)

# Print the contents
# Note: For brevity and clarity, this example only prints the first few items.

print("Image Features (first item):", image_features[0][:10])  # Print first 10 features of the first item for brevity
print("\nProcessed Reviews (first 5):", preprocessed_reviews[:5])
print("\nTF-IDF Scores (first document):", list(tfidf_scores[0].items())[:])  # Print first 5 TF-IDF scores of the first document




In [ ]:

# Initialize ResNet model globally to avoid reloading
resnet_model = ResNet50(weights='imagenet', include_top=False)
model = Model(inputs=resnet_model.input, outputs=resnet_model.output)

def preprocess_image(image_url):
    """Preprocess an image from a URL."""
    try:
        image_url = str(image_url).strip().strip("'").strip('"')
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content)).convert('RGB')
        image = image.resize((224, 224))
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(1.5)
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(1.1)
        return np.array(image)
    except Exception as e:
        print(f"Error processing image from URL {image_url}: {e}")
        return None

def extract_features(image_array):
    """Extract features from an image array using ResNet50."""
    if image_array is not None:
        image_array = preprocess_input(image_array)
        image_array = np.expand_dims(image_array, axis=0)
        features = model.predict(image_array)
        return features.reshape(features.shape[0], -1).tolist()
    return None

def preprocess_text(text):
    """Preprocess text data."""
    text = str(text).lower().translate(str.maketrans('', '', string.punctuation))
    tokens = nltk.word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    return ' '.join(lemmatized_tokens)

def compute_tf(text):
    """Compute term frequency for a document."""
    tf_text = defaultdict(int)
    words = text.split()
    for word in words:
        tf_text[word] += 1
    return {word: count / len(words) for word, count in tf_text.items()}

def compute_idf(documents):
    """Compute inverse document frequency across a corpus."""
    idf_dict = defaultdict(int)
    N = len(documents)
    for document in documents:
        unique_words = set(document.split())
        for word in unique_words:
            idf_dict[word] += 1
    return {word: math.log(N / (count + 1)) for word, count in idf_dict.items()}

def compute_tfidf(documents):
    """Compute TF-IDF scores for all documents in a corpus."""
    idf_dict = compute_idf(documents)
    return [{word: tf * idf_dict[word] for word, tf in compute_tf(document).items()} for document in documents]

# Cosine Similarity for numpy arrays (image features)
def cosine_similarity(vector_a, vector_b):
    # Ensure vectors are 1D
    vector_a = vector_a.flatten()
    vector_b = vector_b.flatten()
    dot_product = np.dot(vector_a, vector_b)
    norm_a = np.linalg.norm(vector_a)
    norm_b = np.linalg.norm(vector_b)
    return dot_product / (norm_a * norm_b) if (norm_a * norm_b) != 0 else 0



# Cosine Similarity for dictionaries (TF-IDF vectors)
def cosine_similarity_text(vec_a, vec_b):
    # Ensure we're working with dense vectors if they're passed as sparse
    if isinstance(vec_a, dict) and isinstance(vec_b, dict):
        intersection = set(vec_a.keys()) & set(vec_b.keys())
        numerator = sum([vec_a[x] * vec_b[x] for x in intersection])

        sum1 = sum([val**2 for val in vec_a.values()])
        sum2 = sum([float(val)**2 for val in vec_b.values()])  # Convert string values to float
        denominator = np.sqrt(sum1) * np.sqrt(sum2)

        return float(numerator) / denominator if denominator != 0 else 0.0
    else:
        raise ValueError("TF-IDF vectors must be of type dict.")



# Find Similar Images
def find_most_similar(features_with_url, input_features, top_k=3):
    similarities = []
    for url, features in features_with_url:
        if features is not None:
            sim = cosine_similarity(input_features, np.array(features))
            similarities.append((url, sim))
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_k]

# Find Similar Reviews
def find_most_similar_reviews(composite_data, input_tfidf, top_k=3):
    """Find and rank reviews based on text similarity."""
    similarities = []
    for item in composite_data:
        sim = cosine_similarity_text(input_tfidf, item['tfidf_score'])
        similarities.append((item['image_url'], sim, item['preprocessed_review']))
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_k]

def save_results(filename, data):
    """Save data to a pickle file."""
    with open(filename, 'wb') as file:
        pickle.dump(data, file)


def main():
    # Prompt user for input
    image_url = input("Enter the image URL: ")
    review_text = input("Enter the review text: ")

    # Load pre-computed data
    with open('/content/drive/MyDrive/IR_Assignments/Assignment_2/image_features.pkl', 'rb') as file:
        image_features_with_url = pickle.load(file)
    with open('//content/drive/MyDrive/IR_Assignments/Assignment_2/composite_data.pkl', 'rb') as file:
        composite_data = pickle.load(file)

    # Preprocess and compute features and TF-IDF for input
    processed_image = preprocess_image(image_url)
    input_image_features = extract_features(processed_image) if processed_image is not None else None
    input_review_tfidf = compute_tfidf([preprocess_text(review_text)])[0]

    # Find the top 3 similar images based on image features
    similar_images = find_most_similar(image_features_with_url, np.array(input_image_features), top_k=3)

    # Saving into pickled file
    save_results('/content/drive/MyDrive/IR_Assignments/Assignment_2/similar_images.pkl', similar_images)

    print("USING IMAGE RETRIEVAL")
    for idx, (img_url, img_sim) in enumerate(similar_images[:3], start=1):
        # Find corresponding review in composite data and calculate text similarity
        review_data = next((item for item in composite_data if item['image_url'] == img_url), None)
        review_text = review_data['preprocessed_review'] if review_data else "Review not found"
        text_sim = cosine_similarity_text(input_review_tfidf, review_data['tfidf_score']) if review_data else 0

        print(f"{idx}) Image URL: {img_url}\nReview: {review_text}\nCosine similarity of images - {img_sim:.3f}\nCosine similarity of text - {text_sim:.4f}\n")

    # Calculate and print composite scores for image retrieval
    composite_image_similarity = sum([sim for _, sim in similar_images]) / len(similar_images)
    composite_text_similarity = sum([text_sim for _, text_sim in similar_images]) / len(similar_images)
    final_composite_similarity = (composite_image_similarity + composite_text_similarity) / 2
    print(f"\nComposite similarity scores of images: {composite_image_similarity:.3f}")
    print(f"Composite similarity scores of text: {composite_text_similarity:.3f}")
    print(f"Final composite similarity score: {final_composite_similarity:.3f}\n")


    # Find the top 3 similar reviews based on text similarity
    similar_reviews = find_most_similar_reviews(composite_data, input_review_tfidf, top_k=3)

    # Saving into pickled file
    save_results('/content/drive/MyDrive/IR_Assignments/Assignment_2/similar_reviews.pkl', similar_reviews)


    print("USING TEXT RETRIEVAL")
    for idx, (img_url, text_sim, review) in enumerate(similar_reviews, start=1):
        # Find corresponding image feature similarity
        img_features = next((features for url, features in image_features_with_url if url == img_url), None)
        if img_features is not None:
            img_sim = cosine_similarity(np.array(input_image_features), np.array(img_features))
        else:
            img_sim = 0  # In case there are no features found

        print(f"{idx}) Image URL: {img_url}\nReview: {review}\nCosine similarity of text - {text_sim:.3f}\nCosine similarity of images - {img_sim:.3f}\n")

    # Calculate and print composite scores for text retrieval
    composite_image_similarity = sum([img_sim for _, img_sim, _ in similar_reviews]) / len(similar_reviews)
    composite_text_similarity = sum([sim for _, sim, _ in similar_reviews]) / len(similar_reviews)
    final_composite_similarity = (composite_image_similarity + composite_text_similarity) / 2
    print(f"\nComposite similarity scores of images: {composite_image_similarity:.3f}")
    print(f"Composite similarity scores of text: {composite_text_similarity:.3f}")
    print(f"Final composite similarity score: {final_composite_similarity:.3f}\n")



if __name__ == "__main__":
    main()



94765736/94765736 [==============================] - 1s 0us/step
Enter the image URL: https://images-na.ssl-images-amazon.com/images/I/712LuK8v2+L._SY88.jpg
Enter the review text: Loving
1/1 [==============================] - 1s 1s/step
USING IMAGE RETRIEVAL
1) Image URL: https://images-na.ssl-images-amazon.com/images/I/712LuK8v2+L._SY88.jpg
Review: hot damn replaced original neck position humbucker 1980 le paul bought new sph90 phat cat difference much louder brighter around sweeter tuneful original used rarely use neck pickup alone ill use almost exclusively though mixing bit bridge humbucker sweet im glad bought made switch p ive problem hum pickup problem probably somewhere amp
Cosine similarity of images - 1.000
Cosine similarity of text - 0.0000

2) Image URL: https://images-na.ssl-images-amazon.com/images/I/71Y9QQZDpvL._SY88.jpg
Review: love great finish quality
Cosine similarity of images - 0.310
Cosine similarity of text - 0.0000

3) Image URL: https://images-na.ssl-images-ama